# Knowledge Structure Graph Generator (Neo4J Implementation)

## 1. DB연결

In [3]:
import pymysql
import pandas as pd
import re

In [4]:
juso_db = pymysql.connect(
    user='root', 
    passwd='1234', 
    host='127.0.0.1', 
    db='test', 
    charset='utf8'
)

In [5]:
cursor = juso_db.cursor(pymysql.cursors.DictCursor)

In [6]:
sql = "SELECT * FROM knowledge_structure;"
cursor.execute(sql)
result = cursor.fetchall()

In [7]:
result = pd.DataFrame(result)
print("------------------------------지식구조 목록 리스트------------------------------")
print(result) #지식구조 목록 리스트화

------------------------------지식구조 목록 리스트------------------------------
       DOC_ID COT_MODE                                KNOWLEDGE_STRUCTURE
0        hemp      scs  sourc,origin,1.0\nsourc,millennium,1.0\nsourc,...
1       kaist      scs  박사,과정,1.0\n공동,카이스트,1.0\n면역,세포,2.19\n기술,개발,2.26...
2          감초      scs  함량,재배,2.9\n효능,역할,3.21\n하루,이상,3.9\n이상,단맛,3.9\n한...
3          녹차      scs  순수,티백,1.52\n제품,제조,2.15\n효과,흥분,2.95\n제품,순수,2.95...
4          대마      scs  마약,법률,3.06\n약물,중독,3.26\n한국,미국,3.87\n필로폰,헤로인,4....
5         바이오      scs  소재,개발,3.07\n분야,성과,3.6\n기술,식품,3.71\n분야,식품,3.79\...
6    인공지능_이루다      scs  제공,특정,2.76\n대화,연인들,3.54\n개발사,여성,3.54\n말투,학습,4....
7   인공지능_이루다2      scs  특정,자사,1.63\n수집,변호사,2.1\n개인정보,유출,2.76\n개인정보,수집,...
8     인공지능_제조      scs  제조,역할,1.0\n산업,제조,1.34\n산업,역할,1.34\n혁신,인공지능,2.3...
9     인공지능_호텔      scs  호텔,서비스,1.99\n코로나,업계,2.76\n제공,정보,2.98\n제공,차량,2....
10          차      scs  남부,지방,2.71\n재배,지역,4.03\n식물,우려,4.81\n남부,지역,4.85...
11          콩      scs  두부,된장,1.8\n고기,짜파

In [8]:
knowledge_structure = result['KNOWLEDGE_STRUCTURE']

## 2. 데이터 선택

#### 작성할 파일 제목 정하기
* file_name = "파일명"

In [9]:
file_name = "바이오"

#### 추출하기 원하는 인덱스 선정

In [10]:
data = knowledge_structure[4] #인덱스만 변경하기

In [11]:
contents = re.split('\n', data)
contents = contents[:-1] #맨 끝 데이터는 공백

In [12]:
#노드 추가
edge_list = list()
node_set = set()

for content in contents:
    value = re.split(',', content) # ,단위로 구분하여 리스트화
    edge_list.append(value) # 이중 리스트
    node_set.add(value[0]) # 노드들을 셋에 추가하여 유니크 값만 저장
    node_set.add(value[1]) 

## 3. Neo4J 연결

In [13]:
from py2neo import Graph, Node, Relationship, RelationshipMatcher

In [14]:
graph = Graph('bolt://neo4j@localhost:7687', auth = ("neo4j", "1234"))

# Authentification Error가 날 경우, 데이터베이스 생성시 설정한 name과 password를 써줍니다.
# graph = Graph('bolt://neo4j@localhost:7687', auth = ("neo4j", "password"))

graph

Graph('bolt://neo4j@localhost:7687')

In [42]:
node2idx = dict() # 각 노드에 인덱스 부여
for idx, node in enumerate(node_set):
    node2idx[node] = idx # 노드가 key, 인덱스가 value
node2idx

{'헤로인': 0,
 '효과': 1,
 '문제': 2,
 '대마': 3,
 '단속': 4,
 '중반': 5,
 '사용': 6,
 '의약품': 7,
 '마약': 8,
 '필로폰': 9,
 '담배': 10,
 '중독': 11,
 '합법': 12,
 '약물': 13,
 '미국': 14,
 '한국': 15,
 '대마초': 16,
 '씨앗': 17,
 '삼베': 18,
 '법률': 19}

In [43]:
nodes_neo = [Node("Keyword", name = node) for node in node2idx.keys()] # Py2neo를 이용해 Node를 선언. Label은 Keyword, Property는 name이다.
nodes_neo

[Node('Keyword', name='헤로인'),
 Node('Keyword', name='효과'),
 Node('Keyword', name='문제'),
 Node('Keyword', name='대마'),
 Node('Keyword', name='단속'),
 Node('Keyword', name='중반'),
 Node('Keyword', name='사용'),
 Node('Keyword', name='의약품'),
 Node('Keyword', name='마약'),
 Node('Keyword', name='필로폰'),
 Node('Keyword', name='담배'),
 Node('Keyword', name='중독'),
 Node('Keyword', name='합법'),
 Node('Keyword', name='약물'),
 Node('Keyword', name='미국'),
 Node('Keyword', name='한국'),
 Node('Keyword', name='대마초'),
 Node('Keyword', name='씨앗'),
 Node('Keyword', name='삼베'),
 Node('Keyword', name='법률')]

- 참고!
    - edge_list는 각 edge를 리스트로 한 이중리스트입니다.
    - 각 edge는 [**노드1, 노드2, 점수**]의 형태로 되어 있습니다.
    - Py2neo 문법 : Relationship(node1, relationship, node2, *relationship_property(옵션)*)
    

In [23]:
tx = graph.begin() # 트랜잭션 객체를 선언하여 시작합

for edge in edge_list: # 각 edge에 대해 관계를 선언
    node1 = nodes_neo[node2idx[edge[0]]]
    node2 = nodes_neo[node2idx[edge[1]]]
    re = Relationship(node1, edge[2], node2, name = "score")
    
    # 생성한 노드 nodes_neo는 위와 같이 node2id를 이용해 인덱스로 접근할 수 있습니다. 
    print(re)
    tx.create(re) # 선언해둔 노드나 관계 객체를 생성
tx.commit() # 트랜잭션을 commit하여 DB에 반영하며 종료됩니다. 선언했던 노드와 관계가 실제로 DB에 반영되는 시기입니다.

if tx.finished() == False:
    print("Transaction is still open")
else:
    print("Transaction has been completed")

(마약)-[:3.06 {name: 'score'}]->(법률)
(약물)-[:3.26 {name: 'score'}]->(중독)
(한국)-[:3.87 {name: 'score'}]->(미국)
(필로폰)-[:4.0 {name: 'score'}]->(헤로인)
(필로폰)-[:4.0 {name: 'score'}]->(중반)
(대마초)-[:4.08 {name: 'score'}]->(한국)
(문제)-[:4.55 {name: 'score'}]->(중반)
(미국)-[:4.55 {name: 'score'}]->(중반)
(문제)-[:4.81 {name: 'score'}]->(의약품)
(대마)-[:4.92 {name: 'score'}]->(효과)
(대마초)-[:4.95 {name: 'score'}]->(사용)
(씨앗)-[:5.04 {name: 'score'}]->(문제)
(대마초)-[:5.22 {name: 'score'}]->(마약)
(마약)-[:5.28 {name: 'score'}]->(약물)
(마약)-[:5.29 {name: 'score'}]->(대마)
(합법)-[:5.38 {name: 'score'}]->(의약품)
(대마)-[:5.54 {name: 'score'}]->(삼베)
(대마초)-[:5.57 {name: 'score'}]->(단속)
(대마초)-[:6.09 {name: 'score'}]->(담배)
Transaction has been completed


In [24]:
graph.run("MATCH(n) RETURN n")

 n                                         
-------------------------------------------
 (_0:Keyword {name: '\ub300\ud654'})       
 (_1:Keyword {name: '\uc5f0\uc778\ub4e4'}) 
 (_2:Keyword {name: '\uac1c\ubc1c\uc0ac'}) 

* 참고
    - run의 경우, 수행과 동시에 자동으로 commit되어 따로 트랜잭션 객체를 생성하거나 commit할 필요가 없습니다.

---

### Neo4J로 생성된 지식구조 확인
이제 다시 Neo4J Browser로 가서 `MATCH(n) RETURN n` 쿼리를 입력하면 그래프 형태로 확인할 수 있습니다.

- Q. Neo4J Browser를 못 찾겠어요!
    - A. Neo4J 데스크탑에서 localhost 클릭하면 창이 새로 뜹니다.